<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

In [1]:
import yaml

# Set `PATH` to include the directory containing TFX CLI.
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [2]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.21.2


# CI/CD for TFX pipelines

In this lab you will walk through authoring of a Cloud Build CI/CD workflow that automatically builds and deploys a TFX pipeline. You will also integrate your workflow with GitHub by setting up a trigger that starts the workflow when a new tag is applied to the GitHub repo hosting the pipeline's code.



## Understanding the Cloud Build workflow
Review the `cloudbuild.yaml` file to understand how the CI/CD workflow is implemented and how environment specific settings are abstracted using **Cloud Build** variables.

The **Cloud Build** CI/CD workflow automates the steps you walked through manually during `lab-02`:
1. Builds the custom TFX image to be used as a runtime execution environment for TFX components and as the AI Platform Training training container.
1. Compiles the pipeline and uploads the pipeline to the KFP environment
1. Pushes the custom TFX image to your project's **Container Registry**

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **TFX CLI**. 


## Configuring environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `GCP_REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.
- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [21]:
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://hostedkfp-default-dcsh2ljqmk'
ENDPOINT = '756f6ab12e557cde-dot-us-central2.pipelines.googleusercontent.com'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

## Creating the TFX CLI builder
### Review the Dockerfile describing the TFX CLI builder

In [4]:
!cat Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-1
COPY requirements.txt .
RUN python -m pip install -U -r requirements.txt

ENTRYPOINT ["tfx"]


In [5]:
!cat requirements.txt

pandas<1.0.0
tfx==0.21.2
kfp==0.2.5


### Build the image and push it to your project's **Container Registry**.

In [6]:
IMAGE_NAME='tfx-cli'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} .

## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the `gcloud builds submit` command.

In [36]:
PIPELINE_NAME='tfx_covertype_training_deployment'
TAG_NAME='test'
TFX_IMAGE_NAME='lab-03-tfx-image'
DATA_ROOT_URI='gs://workshop-datasets/covertype/small'
MODEL_NAME='tfx_covertype_classifier'
PIPELINE_FOLDER='lab-02-tfx-pipeline'
PIPELINE_DSL='pipeline/runner.py'
RUNTIME_VERSION='2.1'
PYTHON_VERSION='3.7'

SUBSTITUTIONS="""
_ENDPOINT={},\
_GCP_REGION={},\
_ARTIFACT_STORE_URI={},\
_TFX_IMAGE_NAME={},\
_DATA_ROOT_URI={},\
_MODEL_NAME={},\
TAG_NAME={},\
_PIPELINE_FOLDER={},\
_PIPELINE_DSL={},\
_PIPELINE_NAME={},\
_RUNTIME_VERSION={},\
_PYTHON_VERSION={}
""".format(ENDPOINT, 
           GCP_REGION, 
           ARTIFACT_STORE_URI, 
           TFX_IMAGE_NAME,
           DATA_ROOT_URI,
           MODEL_NAME,
           TAG_NAME, 
           PIPELINE_FOLDER,
           PIPELINE_DSL,
           PIPELINE_NAME,
           RUNTIME_VERSION,
           PYTHON_VERSION
           ).strip()

In [37]:
!gcloud builds submit .. --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 41 file(s) totalling 213.6 KiB before compression.
Uploading tarball of [..] to [gs://mlops-workshop_cloudbuild/source/1585164063.76-623be71616554c8c8f44b4692acc9ba5.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-workshop/builds/d8d6e6b3-2db7-4cfe-ac1a-42932d975619].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/d8d6e6b3-2db7-4cfe-ac1a-42932d975619?project=745302968357].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d8d6e6b3-2db7-4cfe-ac1a-42932d975619"

FETCHSOURCE
Fetching storage object: gs://mlops-workshop_cloudbuild/source/1585164063.76-623be71616554c8c8f44b4692acc9ba5.tgz#1585164064157796
Copying gs://mlops-workshop_cloudbuild/source/1585164063.76-623be71616554c8c8f44b4692acc9ba5.tgz#1585164064157796...
/ [1 files][ 50.1 KiB/ 50.1 KiB]                                                
Operation completed over 1 objects/50.1 KiB.                   

## Setting up GitHub integration

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Create a fork of this repo
[Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork this repo

### Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location|/ workshops/kfp-caip-sklearn/lab-03-tfx-cicd/cloudbuild.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_BASE_IMAGE_NAME|base_image|
|_COMPONENT_URL_SEARCH_PREFIX|https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/|
|_ENDPOINT|[Your inverting proxy host]|
|_PIPELINE_DSL|covertype_training_pipeline.py|
|_PIPELINE_FOLDER|workshops/kfp-caip-sklearn/lab-03-kfp-cicd/pipeline|
|_PIPELINE_NAME|covertype_training_deployment|
|_PIPELINE_PACKAGE|covertype_training_pipeline.yaml|
|_PYTHON_VERSION|3.7|
|_RUNTIME_VERSION|1.15|
|_TRAINER_IMAGE_NAME|trainer_image|

### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```
